Lien d'explication de ce notebook:

https://www.youtube.com/watch?v=iRunifGSHFc&list=PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK&index=5 

In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [4]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")

In [5]:
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/luca/teaching/simplon/IA_2024/teaching-resources/Ressources/MLOps/mlops-zoomcamp/01-intro/mlruns/1', creation_time=1730728484294, experiment_id='1', last_update_time=1730728484294, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [6]:
def read_dataframe(filename):

    df = pd.read_csv(filename)
    df['lpep_pickup_datetime'] = pd.to_datetime(df.lpep_pickup_datetime)
    df['lpep_dropoff_datetime'] = pd.to_datetime(df.lpep_dropoff_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

## Evaluation sur un jeu de validation

In [7]:
df_train = read_dataframe('../data/green_tripdata_2021-01.csv')
df_val = read_dataframe('../data/green_tripdata_2021-02.csv')

/tmp/ipykernel_4317/1721828809.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [8]:
len(df_train), len(df_val)

(73908, 61921)

In [9]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [10]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [11]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

np.float64(7.75871521021275)

In [13]:
# with open('models/lin_reg.bin', 'wb+') as f_out:
#     pickle.dump((dv, lr), f_out)

## With MLFlow

In [14]:
# with mlflow.start_run():

#     mlflow.set_tag('model', 'lasso-regression')
#     mlflow.set_tag('developer', 'luca')
#     mlflow.log_param('train-data-path', '../data/green_tripdata_2021-01.csv')
#     mlflow.log_param('valid-data-path', '../data/green_tripdata_2021-02.csv')

#     alpha = 0.01
#     mlflow.log_param('alpha', alpha)

#     lr = Lasso(alpha)

#     lr.fit(X_train, y_train)

#     y_pred = lr.predict(X_val)

#     rmse = root_mean_squared_error(y_val, y_pred)

#     mlflow.log_metric('val-rmse', rmse)

#     with open('models/lasso_reg.bin', 'wb+') as f_out:
#         pickle.dump((dv, lr), f_out)

#     mlflow.log_artifact(local_path='models/lasso_reg.bin', artifact_path="models_pickle")


## With Parameter Tuning

In [15]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [16]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [17]:
from sklearn.metrics import mean_squared_error


# def objective(params):
#     with mlflow.start_run():
#         mlflow.set_tag('model', 'xgboost')
#         mlflow.log_params(params)
#         booster = xgb.train(
#             params=params,
#             dtrain=train,
#             num_boost_round=20,
#             evals=[(valid, 'validation')],
#             early_stopping_rounds=10
#         )
#         y_pred = booster.predict(valid)
#         rmse = mean_squared_error(y_val, y_pred, squared=False)
#         mlflow.log_metric("rmse", rmse)


#     return {'loss': rmse, 'status': STATUS_OK}

In [18]:
# search_space = {
#     'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
#     'learning_rate': hp.loguniform('learning_rate', -3, 0),
#     'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
#     'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
#     'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
#     'objective': 'reg:linear',
#     'seed': 42
# }


# best_result = fmin(
#     fn=objective,
#     space=search_space,
#     algo=tpe.suggest,
#     max_evals=10,
#     trials=Trials()
# )

## With Autolog

In [ ]:
best_params = {
    'learning_rate': 0.09585355369315604,
    'max_depth': 30,
    'min_child_weight': 1.060597050922164,
    'objective': 'reg:squarederror',
    'reg_alpha': 0.018060244040060163,
    'reg_lambda': 0.011658731377413597,
    'seed': 42
}

# does not work for some reason (likely a compatibility issue between mlflow and xgboost)
# mlflow.xgboost.autolog(extra_tags={'model': 'xgboost'})

# with mlflow.start_run():
#     booster = xgb.train(
#         params=best_params,
#         dtrain=train,
#         num_boost_round=20,
#         evals=[(valid, 'validation')],
#         early_stopping_rounds=10
#     )
#     mlflow.xgboost.autolog(disable=True)

#     X_val_xgb = xgb.DMatrix(X_val)
#     y_pred = xgb.predict(X_val_xgb)
#     rmse = mean_squared_error(y_val, y_pred, squared=False)
#     mlflow.log_metric("validation-rmse", rmse)

#     with open("models/preprocessor.b", "wb") as f_out:
#         pickle.dump(dv, f_out)
#     mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")


In [ ]:
mlflow.sklearn.autolog()
with mlflow.start_run():
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlmodel = LinearRegression()
    mlmodel.fit(X_train, y_train)

    mlflow.sklearn.autolog(disable=True)

    y_pred = mlmodel.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("validation-rmse", rmse)


2024/11/04 19:38:36 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/home/luca/.pyenv/versions/3.10.6/envs/mlops_env/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [21]:
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()
with mlflow.start_run():
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlmodel = LinearSVR()
    mlmodel.fit(X_train, y_train)

    mlflow.sklearn.autolog(disable=True)

    y_pred = mlmodel.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("validation-rmse", rmse)


2024/11/04 19:41:51 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/home/luca/.pyenv/versions/3.10.6/envs/mlops_env/lib/python3.10/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/luca/.pyenv/versions/3.10.6/envs/mlops_env/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [19]:
from sklearn.linear_model import Ridge

mlflow.sklearn.autolog()
with mlflow.start_run():
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlmodel = Ridge(alpha=0.1)
    mlmodel.fit(X_train, y_train)

    mlflow.sklearn.autolog(disable=True)

    y_pred = mlmodel.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("validation-rmse", rmse)


2024/11/04 20:03:08 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/home/luca/.pyenv/versions/3.10.6/envs/mlops_env/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
